*von Benedikt Funke*

### Einfügen von Stationsdaten in die MongoDB

Eine entscheidende Datengrundlage bilden die Stationsdaten der einzelnen Wetterstationen. Anhand derer Koordinaten wird in einem späteren Schritt die Projektion auf eine Deutschlandkarte vorgenommen. Außerdem wird über die `STATION_ID` die Zuordnung der Temperaturen zu den richtigen Station vorgenommen.

Um eine möglichst einheitliche und zuverlässige Datenquelle zu verwenden, werden die offiziellen Wetterdaten des Deutschen Wetterdienstes verwendet. Dieser stellt unter seiner OpenData-Strategie eine Vielzahl von aktuellen und historischen Wetterdaten zur Verfügung. Diese Daten können in python mithilfe der `wetterdienst`-Bibliothek (siehe [github](https://github.com/earthobservations/wetterdienst)) abgerufen werden.

Alle Stationen, die für die Auswertung der Temperaturen genutzt werden, müssen in dem Zeitraum vom 01.07.2020 bis zum 31.08.2020 stündlich Daten geliefert haben. Dies wird durch die verschiedenen Parametern beim Aufrufen der `DWDObservationSites`-Klasse erreicht. (siehe unten)

Die erhaltenen Daten befinden sich in einem `pandas Dataframe` und werden für die Speicherung in der Datenbank in ein `Dictionary` umgewandelt. Dieses wird anschließend direkt mithilfe der `pymongo`-Bibliothek in die Datenbank importiert. Vorher werden jedoch noch eventuell vorhandene alte Daten gelöscht.

In [ ]:
from wetterdienst.dwd.observations import DWDObservationSites, DWDObservationParameterSet, DWDObservationPeriod, DWDObservationResolution
from pymongo import MongoClient

sites = DWDObservationSites(
    parameter_set=DWDObservationParameterSet.TEMPERATURE_AIR,
    resolution=DWDObservationResolution.HOURLY,
    start_date="2020-07-01",
    end_date="2020_08-31",
    period=DWDObservationPeriod.HISTORICAL
)
df = sites.all()
df.head()
df_dict = df.to_dict("records")

client = MongoClient('mongodb://localhost:27017/')
stations = client.bda.stations
stations.delete_many({})

stations.insert_many(df_dict)
client.close()